<a href="https://colab.research.google.com/github/YolandaNkalashe25/COS801/blob/main/COS_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install torch torchvision
import torch
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [4]:
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

In [2]:
Fashion_MNIST_DF=pd.read_csv('/content/drive/My Drive/Colab Notebooks/COS801/fashion-mnist_train.csv')

In [3]:
Fashion_MNIST_DF.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211.0,220.0,214.0,74.0,0.0,255.0,222.0,128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,12.0,0.0,0.0,40.0,134.0,162.0,191.0,214.0,163.0,146.0,165.0,79.0,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171.0,249.0,207.0,197.0,202.0,45.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,21.0,25.0,69.0,52.0,45.0,74.0,39.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230.0,237.0,229.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0,116.0,112.0,136.0,147.0,144.0,121.0,102.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
col=Fashion_MNIST_DF.columns

In [7]:
Fashion_MNIST_DF=Fashion_MNIST_DF.replace('?',np.nan)

In [8]:
Fashion_MNIST_DF=Fashion_MNIST_DF.dropna()

In [ ]:
import seaborn as sns
sns.countplot(Fashion_MNIST_DF.label)

In [10]:
df_features=Fashion_MNIST_DF[col[1:]]
df_target=Fashion_MNIST_DF[col[0]]

In [11]:
#standardize the value.
x=df_features.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_features= pd.DataFrame(x_scaled)


In [13]:
df_features.columns=col[1:]

In [ ]:
df_features.head()

In [16]:
# Split data into training and validation set.
X_train, x_val, Y_train, y_val=train_test_split(df_features,df_target,test_size=0.2,random_state=7)

In [17]:
X_train = torch.from_numpy(X_train.to_numpy()).float()
Y_train = torch.squeeze(torch.from_numpy(Y_train.to_numpy()).float())
x_val = torch.from_numpy(x_val.to_numpy()).float()
y_val = torch.squeeze(torch.from_numpy(y_val.to_numpy()).float())


In [18]:
print(X_train.shape, Y_train.shape)
print(x_val.shape, y_val.shape)

torch.Size([8713, 784]) torch.Size([8713])
torch.Size([2179, 784]) torch.Size([2179])


In [13]:
input_features=784
output_size=10
hidden_size=0

In [14]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_features, output_size)

  def forward(self, x):
    x = F.log_softmax(self.fc1(x),dim=1)
    return x

In [15]:
model=Net()

In [16]:
model

Net(
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [46]:
Y_train=Y_train.type(torch.LongTensor)
y_val=y_val.type(torch.LongTensor)

In [17]:
#Optimizer to update the weights during backpropagation use stochatic gradient descent optimizer. We instantiate with our model parameters
optimizer = optim.SGD(model.parameters(),lr=0.01)

Loss_fn=nn.NLLLoss()


In [48]:
ephoch_data=[]
ephoc=10001

In [56]:
import tensorflow as tf

In [63]:
y_val.size()[0]

2179

In [ ]:
#Start training for a 1000 ephocs.
#for every ephoc we zero out our gradients so that we have fresh gradients:
for ephoch in range(1,ephoc):

  optimizer.zero_grad()
  Ypred=model(X_train)

  #We then perform a backward pass.The model only has a function for a forward pass.

  loss=Loss_fn(Ypred, Y_train)
  loss.backward()  ## Calculate the gradients once loss has been calculated.

  #Once gradients are calculated we need to update our parameters:

  optimizer.step()

  ##For every ephoc we calculate the loss on the validation set to see how our data performs on the validation set:
  Ypred_validation=model(x_val)
  loss_validation=Loss_fn(Ypred_validation, y_val)
  # we do not call the backward function when testing on our validation set because we do not use this loss value to update the weights.

  _,pred=Ypred_validation.data.max(1) # Our prediction is the value with the highest probabilty

  #For every ephoch we print out the accuracy of the validation set and the loss.
  accuracy =pred.eq(y_val.data).sum().item()/y_val.size()[0]
  ephoch_data.append([ephoch,loss.data.item(),loss_validation.data.item(),accuracy])
  
  #for every 100 ephocs print out the data to screen

  if ephoch % 100==0:
    print('ephoc- %d (%d%%) train loss - %.2f validation loss - %.2f accuracy - %.4f'\
             % (ephoch, ephoch/150 * 10, loss.data.item(),loss_validation.data.item(),accuracy))


In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
Fashion_MNIST_DF=pd.read_csv('/content/drive/My Drive/Colab Notebooks/COS801/fashion-mnist_train.csv')

In [6]:
Fashion_MNIST_DF.shape

(60000, 785)

In [7]:
trainset = torch.utils.data.DataLoader(Fashion_MNIST_DF, batch_size=50, shuffle=True, num_workers=2)


In [8]:
trainset.dataset.shape

(60000, 785)

In [10]:
trainset.dataset['label']


0        2
1        9
2        6
3        0
4        3
        ..
59995    9
59996    1
59997    8
59998    8
59999    7
Name: label, Length: 60000, dtype: int64